# Day 3

Goals:
- Corner and Edge Detection (15 min)
- Template matching (15 min)
- [  ] Show video of tracking and run demo (15 min)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
# %matplotlib inline

## Useful functions

In [ ]:
def show_rgb(img):
    # Will expect RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title('Look weird if you entered BGR...')
    #plt.show()
    
def show_bgr(img):
    # Will expect RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    #plt.show()
    
def show_cv2(img):
    # Input should be BGR
    cv2.imshow('image',img)
    
    cv2.waitKey(0) # Closes with any key
    cv2.destroyAllWindows()
    
def display(img,cmap=None):
    # Create with larger size
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap)
    
    #from skimage.exposure import rescale_intensity

def convolve(image, kernel):
    # 1) Grab the spatial dimensions of the image, along with
    # the spatial dimensions of the kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    
    # 2) Allocate memory for the output image, taking care to
    # "pad" the borders of the input image so the spatial
    # size (i.e., width and height) are not reduced
    # padding: if ur image is 10x10 and kernel is 3x3, padded image should be 12x12
    pad = (kW - 1) // 2 # if kernel width is kW=3, this will return  pad=1. "//" returns the quocient
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad,cv2.BORDER_REPLICATE)
    # Output image
    output = np.zeros((iH, iW), dtype="float32")
    
    # 3) Loop over the input image, "sliding" the kernel across
    # each (x, y)-coordinate from left-to-right and top to
    # bottom
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # 4) Extract the ROI of the image by extracting the
            # *center* region of the current (x, y)-coordinates
            # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
 
            # 5) Perform the actual convolution by taking the
            # element-wise multiplicate between the ROI and
            # the kernel, then summing the matrix
            k = (roi * kernel).sum()
 
            # 6) Store the convolved value in the output (x,y)-
            # coordinate of the output image
            output[y - pad, x - pad] = k
            
    # 7) Rescale the output image to be in the range [0, 255]
    #output = rescale_intensity(output, in_range=(0, 255))
    #output = (output * 255).astype("uint8")
 
    # return the output image
    return output

## Edge Detection

You can try corner and edge detection using the convolution function you wrote and/or use the built-in OpenCV functions.

In [ ]:
# Open image
chess = cv2.imread('chess.png')
chess0 = cv2.imread('chess.png',0)
#chess = cv2.cvtColor(flat_chess,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(7,10))
plt.subplot(1,2,1)
show_bgr(chess)
plt.subplot(1,2,2)
plt.imshow(chess0,cmap='gray')

In [ ]:
# construct the Laplacian kernel used to detect edge-like
# regions of an image
laplacian = np.array((
[0, 1, 0],
[1, -4, 1],
[0, 1, 0]), dtype="int")
 
# construct the Sobel x-axis kernel
sobelX = np.array((
[-1, 0, 1],
[-2, 0, 2],
[-1, 0, 1]), dtype="int")
 
# construct the Sobel y-axis kernel
sobelY = np.array((
[-1, -2, -1],
[0, 0, 0],
[1, 2, 1]), dtype="int")

In [ ]:
# Convolve using sobelY: why we have this result?
out = convolve(chess0,sobelY)
plt.imshow(out,cmap='gray')

In [ ]:
# Convolve using X: : why we have this result?
out = convolve(chess0,sobelX)
plt.imshow(out,cmap='gray')

In [ ]:
# What about the other edges? How to get them?
# You can actually do that by modifying the previous kernels.
# Hint: Just invert them ;)

# construct the Sobel x-axis kernel (inverted)
sobelXi = np.array((
[1, 0, -1],
[2, 0, -2],
[1, 0, -1]), dtype="int")
 
# construct the Sobel y-axis kernel (inverted)
sobelYi = np.array((
[1, 2, 1],
[0, 0, 0],
[-1,-2,-1]), dtype="int")

In [ ]:
# Try those inverted kernels: did you get what you wanted?
out = convolve(chess0,sobelXi)
plt.imshow(out,cmap='gray')
out = convolve(chess0,sobelYi)
plt.imshow(out,cmap='gray')

## Corner Detection

### How to detect corner using the kernels?
- SOLUTION: Corner detection is where the X-grad and Y-grad meet! That's the intuition behind most popular method Harris-Corner detection (Shi-Tomasi detector)

**cornerHarris Function**

*  src: Input single-channel 8-bit or floating-point image.
*  dst: Image to store the Harris detector responses. It has the type CV_32FC1 and the same size as src .
*  blockSize: Neighborhood size (see the details on #cornerEigenValsAndVecs ).
*  ksize: Aperture parameter for the Sobel operator.
*  k: Harris detector free parameter. See the formula in DocString

In [ ]:
# Convert Gray Scale Image to Float Values
gray = np.float32(chess0)

# Corner Harris Detection
dst = cv2.cornerHarris(src=gray,blockSize=2,ksize=3,k=0.04)

# result is dilated for marking the corners, not important to actual corner detection
# this is just so we can plot out the points on the image shown
#dst = cv2.dilate(dst,None)

plt.imshow(dst,cmap='gray')

In [ ]:
# Draw the detected corners in the original image
chess = cv2.cvtColor(chess,cv2.COLOR_BGR2RGB)
# Threshold for an optimal value, it may vary depending on the image.
chess[dst>0.01*dst.max()]=[255,0,0]

plt.imshow(chess)

## Template Matching

Template matching is the most fundamental method for object detection. The idea is simple. Given a template image, find it in the image! How would you implement it? By subtraction? That's possible, but it would affected by intensity variations. Most common way is to use correlation metric. However, you can actually even use the convolution function you just coded!

Before you try to do it, let's illustrate the intuition behind using convolution for template matching.

In [ ]:
## Create an image of a 3x3 square
img = np.zeros([10,10])
img[4:7,4:7] = 255
plt.imshow(img,cmap='gray')

In [ ]:
# What happens if you convolve this image?
convolve(img,np.ones([3,3])).astype(int)

Exactly! Whenever you have a part of the image matching the kernel image, it will generate the highest values.

### [ ] EXERCISE: Write a code to find Waldo, given the image and the actual image of Waldo.

In [ ]:
# Given Where is Waldo image
where = cv2.imread('whereiswaldo1.jpg')
where_rgb = cv2.cvtColor(where,cv2.COLOR_BGR2RGB)
display(where_rgb)

In [ ]:
# Open the template
waldo = cv2.imread('waldo.jpg')
waldo_rgb = cv2.cvtColor(waldo,cv2.COLOR_BGR2RGB)
display(waldo_rgb)

**SOLUTION**: You can try using your convolution function, 
however, you might have to adapt it to accept non-square shape kernels. You can try any other approach as well. To keep it simple, let's use the built-in function for template matching.


In [ ]:
# Open the image. You can just work with BGR images.
where = cv2.imread('whereiswaldo1.jpg')
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
method = eval(methods[0])
res = cv2.matchTemplate(where,waldo,method)
# Show the output image: how does it look like? Can you find Waldo there?
plt.imshow(res,cmap='gray')

### Drawing a rectangle in the max point
To really know whether the method is working, we can draw a box around the point with the maximum value.

In [ ]:
# get dim 
height, width,channels = waldo.shape

# Grab the Max and Min values, plus their locations
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

# Set up drawing of Rectangle
top_left = max_loc
# Assign the Bottom Right of the rectangle
bottom_right = (top_left[0] + width, top_left[1] + height)

# Draw the Red Rectangle
cv2.rectangle(where,top_left, bottom_right,color=(0,255,0),thickness=3)

# save image
status = cv2.imwrite('found_WALDO.png',where)
print("Image written to file-system : ",status)

In [ ]:
# Check the saved image or show check it here. Did it work?
show_cv2(where)

### [ ] EXERCISE (EXTRA)
Use the template matching approach to find the most red car in the previous exercise.